In [10]:

import requests
import pandas as pd
import mysql.connector
import sqlalchemy
from sqlalchemy import create_engine

api_key = "Enter your API KEY"   # API key
def create_database_schema():
    connection = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = "jeya",
    auth_plugin='mysql_native_password'

    )

    con = connection.cursor()
    con.execute("CREATE DATABASE bookscape")
    connection.commit()
    con.execute('use bookscape')
    host = "localhost"
    user = "root"
    password = "jeya"
    port = 3306
    database_ip='127.0.0.1'
    database = "bookscape"
# Create a SQLAlchemy engine
    my_conn = create_engine('mysql+pymysql://{0}:{1}@{2}:{3}/{4}'.format(user,password,host,port,database))

    table_name = "book_data"
# Load the DataFrame into the SQL database table 'book_data'
    df2.to_sql(table_name,my_conn,if_exists="replace",index=False) #['fail,replace,append']
    print("successfully inserted in book_data")
    con.execute('alter table book_data modify column new_id varchar(100)')
    #con.execute('alter table book_data add primary key(book_id)')
    connection.commit()
    #con.execute("update book_data set newid=concat(book_id,'+', search_key)")
    #connection.commit()
    con.execute('alter table book_data add primary key(new_id)')  
    connection.commit()
 # Load the DataFrame into the SQL database table 'authors_publishers'   
    table_name = "authors_publishers"
    df3.to_sql(table_name,my_conn,if_exists="replace",index=False) #['fail,replace,append']
    print("successfully inserted in authors_publishers")
    con.execute('alter table authors_publishers modify column new_id varchar(100)')
    connection.commit() 
    con.execute('alter table authors_publishers add foreign key(new_id) references book_data(new_id)')
    connection.commit()
# Load the DataFrame into the SQL database table 'rating'   
    table_name = "rating"
    df4.to_sql(table_name,my_conn,if_exists="replace",index=False) #['fail,replace,append']
    print("successfully inserted in rating")
    con.execute('alter table rating modify column new_id varchar(100)')
    connection.commit() 
    con.execute('alter table rating add foreign key(new_id) references book_data(new_id)')
    connection.commit()   
# Load the DataFrame into the SQL database table 'price'   
    table_name = "price"
    df5.to_sql(table_name,my_conn,if_exists="replace",index=False) #['fail,replace,append']
    print("successfully inserted in price")
    con.execute('alter table price modify column new_id varchar(100)')
    connection.commit() 
    con.execute('alter table price add foreign key(new_id) references book_data(new_id)')
    connection.commit() 
    con.close()
def get_book_data(search_query, max_results=1000):
    """
    Fetches book data from the Google Books API.

    Args:
        search_query: The search term for books.
        max_results: The maximum number of results to retrieve (default is 5000).

    Returns:
        A pandas DataFrame containing book information, or None if an error occurs.
    """

    base_url = "https://www.googleapis.com/books/v1/volumes"
    params = {
        "q": search_query,
        "key": api_key,
        "maxResults": 40
    }

    all_book_data = []
    startIndex = 0
    total_items = 0

    try:
        while startIndex < min(max_results, 1000) and (startIndex < total_items or total_items == 0):
       # while startIndex <= max_results:
            params["startIndex"] = startIndex
            #print('param',params)
            response = requests.get(base_url, params=params)
            response.raise_for_status()
            data = response.json()

            if 'items' not in data:
                #print("No items found in API response.")
                break

            items = data['items']
            #t(items)
            total_items = data.get("totalItems", 0)
            #print('total',len(items))

            for item in items:
                volume_info = item.get('volumeInfo', {})
                sale_info = item.get('saleInfo', {})

                book_info = {
                    'book_id': item.get('id', ''),
                    'search_key': search_query,
                    'book_title': volume_info.get('title', ''),
                    'book_subtitle': volume_info.get('subtitle', ''),
                    #'volume_id': volume_info.get('id', ''),
                    'book_authors': ", ".join(volume_info.get('authors', [])),
                    'book_description': volume_info.get('description', ''),
                    'publisher': volume_info.get('publisher', ''),
                    'industryIdentifiers': ", ".join([identifier.get('identifier', '') for identifier in volume_info.get('industryIdentifiers', [])]),
                    'text_readingModes': volume_info.get('readingModes', {}).get('text', False),
                    'image_readingModes': volume_info.get('readingModes', {}).get('image', False),
                    'pageCount': volume_info.get('pageCount', 0),
                    'categories': ", ".join(volume_info.get('categories', [])),
                    'language': volume_info.get('language', ''),
                    'imageLinks': str(volume_info.get('imageLinks', {})),
                    'ratingsCount': volume_info.get('ratingsCount', 0),
                    'averageRating': volume_info.get('averageRating', 0),
                    'country': sale_info.get('country', ''),
                    'saleability': sale_info.get('saleability', ''),
                    'isEbook': sale_info.get('isEbook', False),
                    'amount_listPrice': sale_info.get('listPrice', {}).get('amount', 0),
                    'currencyCode_listPrice': sale_info.get('listPrice', {}).get('currencyCode', ''),
                    'amount_retailPrice': sale_info.get('retailPrice', {}).get('amount', 0),
                    'currencyCode_retailPrice': sale_info.get('retailPrice', {}).get('currencyCode', ''),
                    'buyLink': sale_info.get('buyLink', ''),
                    'year': volume_info.get('publishedDate', '').split('-')[0] if volume_info.get('publishedDate', '') else '',
                    'new_id': item.get('id', '')+'+'+search_query

                }
               
                all_book_data.append(book_info)


            startIndex += 40

        if not all_book_data:
            print("No book data found for the search query.")
            return None
        return pd.DataFrame(all_book_data)

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None


df1=pd.DataFrame()
search_term = ["maths","data science","English Literature","Python programming"]
#search_term="data science"
for i in search_term:
    #if i == 'maths':
    print('search',i)
    df = get_book_data(i, max_results=1000)
        #print(df)
    if df is not None:
        df1=pd.concat([df1,df],axis=0,ignore_index=True)
        
        print(f"Total books found: {len(df1)}")
    
if df1 is not None:  
    
    df1['averageRating']=df1['averageRating'].astype(float)
    df1['ratingsCount']=df1['ratingsCount'].astype(float)
    df1['amount_listPrice']=df1['amount_listPrice'].astype(float)
    df1['amount_retailPrice']=df1['amount_retailPrice'].astype(float)
    #print(df1.info())   
    #print(df1.tail(10))
    df1=df1.drop_duplicates(keep=False)
    df2=df1.loc[:,['book_id','search_key','book_title','book_subtitle','book_description','industryIdentifiers','text_readingModes','image_readingModes','pageCount','categories','language','imageLinks','country','saleability','isEbook','buyLink','year','new_id']]
    df3=df1.loc[:,['new_id','book_authors','publisher']]
    df4=df1.loc[:,['new_id','ratingsCount','averageRating']]
    df5=df1.loc[:,['new_id','amount_listPrice','currencyCode_listPrice','amount_retailPrice','currencyCode_retailPrice']]
    create_database_schema() 
    
    

search maths
Total books found: 918
search data science
Total books found: 1678
search English Literature
Total books found: 2534
search Python programming
Total books found: 3289
successfully inserted in book_data
successfully inserted in authors_publishers
successfully inserted in rating
successfully inserted in price
